In [1]:
import openai
print(openai.__version__)

0.27.9


In [2]:
#pip install openai==0.27.9


In [3]:
from dotenv import dotenv_values
config = dotenv_values(".env")
openai.api_key = config['OPEN_API_KEY']
assert openai.api_key.startswith('sk-'), 'Error loading the API key. The API key should start with "sk-""'

In [4]:
train = openai.File.create(
    file=open('train.jsonl','rb'),
    purpose='fine-tune'
)
train

<File file id=file-9HZZAgjeb6BRXEN7PyUVSqWG at 0x105662f20> JSON: {
  "object": "file",
  "id": "file-9HZZAgjeb6BRXEN7PyUVSqWG",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 5936,
  "created_at": 1696772960,
  "status": "uploaded",
  "status_details": null
}

In [5]:
val = openai.File.create(
    file=open('eval.jsonl','rb'),
    purpose='fine-tune'
)
val

<File file id=file-gWz9TPnlN0fMN1WG89GKOxHw at 0x103c622f0> JSON: {
  "object": "file",
  "id": "file-gWz9TPnlN0fMN1WG89GKOxHw",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 5730,
  "created_at": 1696772963,
  "status": "uploaded",
  "status_details": null
}

In [8]:
print(train['id'],val['id'])


file-9HZZAgjeb6BRXEN7PyUVSqWG file-gWz9TPnlN0fMN1WG89GKOxHw


In [7]:
response = openai.FineTuningJob.create(
    training_file=train['id'],
    validation_file=val['id'],
    model='gpt-3.5-turbo'
)
response

<FineTuningJob fine_tuning.job id=ftjob-RAHMrZPxS7THV4A5OwOYgAVw at 0x105663010> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-RAHMrZPxS7THV4A5OwOYgAVw",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1696772969,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-7viRJjzvHzBH6Vg9YbLiLe9C",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-gWz9TPnlN0fMN1WG89GKOxHw",
  "training_file": "file-9HZZAgjeb6BRXEN7PyUVSqWG",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}

In [9]:
job_id = response["id"]
job_id

'ftjob-RAHMrZPxS7THV4A5OwOYgAVw'

In [16]:
resp = openai.FineTuningJob.retrieve(job_id)
print(resp)

{
  "object": "fine_tuning.job",
  "id": "ftjob-RAHMrZPxS7THV4A5OwOYgAVw",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1696772969,
  "finished_at": 1696773347,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::87OY0skz",
  "organization_id": "org-7viRJjzvHzBH6Vg9YbLiLe9C",
  "result_files": [
    "file-H0FkrvJhI3RTxGHoqvSsfE3V"
  ],
  "status": "succeeded",
  "validation_file": "file-gWz9TPnlN0fMN1WG89GKOxHw",
  "training_file": "file-9HZZAgjeb6BRXEN7PyUVSqWG",
  "hyperparameters": {
    "n_epochs": 4
  },
  "trained_tokens": 4048,
  "error": null
}


In [17]:
resp = openai.FineTuningJob.list_events(id=job_id,limit=10)
print(resp)

{
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-Iq1w1K4CuYmhJqIZeCE1n15b",
      "created_at": 1696773351,
      "level": "info",
      "message": "The job has successfully completed",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-RSEC2IaPfikcEkFX9GWOqQ1r",
      "created_at": 1696773349,
      "level": "info",
      "message": "New fine-tuned model created: ft:gpt-3.5-turbo-0613:personal::87OY0skz",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-9qRSlEOXcXlvCyZ3Pjgjj132",
      "created_at": 1696773327,
      "level": "info",
      "message": "Step 91/100: training loss=0.36, validation loss=0.14",
      "data": {
        "step": 91,
        "train_loss": 0.3607913553714752,
        "valid_loss": 0.1406932830810547,
        "train_mean_token_accuracy": 0.9166666865348816,
        "valid_mean_t

In [18]:
events = resp['data']
events.reverse()
for event in events:
    print(event['message'])

Step 21/100: training loss=0.39, validation loss=0.21
Step 31/100: training loss=0.00, validation loss=0.23
Step 41/100: training loss=0.08, validation loss=0.16
Step 51/100: training loss=0.10, validation loss=0.21
Step 61/100: training loss=0.32, validation loss=2.06
Step 71/100: training loss=0.11, validation loss=0.00
Step 81/100: training loss=0.00, validation loss=0.18
Step 91/100: training loss=0.36, validation loss=0.14
New fine-tuned model created: ft:gpt-3.5-turbo-0613:personal::87OY0skz
The job has successfully completed


In [19]:
completion = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role':'system','content':'You are a smart assistant that reply with Latin proverbs'},
        {'role': 'user','content': 'What is friendship?'}
    ]
)

In [20]:
completion.choices[0].message

<OpenAIObject at 0x1056d4cc0> JSON: {
  "role": "assistant",
  "content": "Amicitia est virtus et vitium non deserens. (Friendship is a virtue that never abandons vice.)"
}

In [22]:
completion = openai.ChatCompletion.create(
    model='ft:gpt-3.5-turbo-0613:personal::87OY0skz',
    messages=[
        {'role':'system','content':'You are a smart assistant that reply with Latin proverbs'},
        {'role': 'user','content': 'What is friendship?'}
    ]
)

In [23]:
completion.choices[0].message

<OpenAIObject at 0x1056ecd60> JSON: {
  "role": "assistant",
  "content": "Amicus certus in re incerta cernitur."
}